In [1]:
"""utility and helper functions / classes."""
import json
import logging
import os
import random
from typing import Tuple

import numpy as np
import torch
from sklearn.metrics import f1_score
from tqdm import tqdm
from transformers import AutoTokenizer

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)


def make_MELD_IEMOCAP():

    SEED = 42
    ratios = {"train": 0.9, "val": 0.1, "test": 0}

    assert sum(list(ratios.values())) == 1

    utterance_ordered = {}

    with open(f"./MELD/utterance-ordered.json", "r") as stream:
        utterance_ordered["MELD"] = json.load(stream)

    with open(f"./IEMOCAP/utterance-ordered.json", "r") as stream:
        utterance_ordered["IEMOCAP"] = json.load(stream)

    diaids_merged = []

    for DATASET in ["MELD", "IEMOCAP"]:
        for SPLIT in ["train", "val", "test"]:
            diaids = list(utterance_ordered[DATASET][SPLIT].keys())
            for diaid in diaids:
                diaids_merged.append(f"{DATASET}/{SPLIT}/{diaid}")

    random.seed(SEED)
    random.shuffle(diaids_merged)

    train_idx = int(len(diaids_merged) * ratios["train"])
    val_idx = int(len(diaids_merged) * (ratios["train"] + ratios["val"]))

    diaids_train = diaids_merged[:train_idx]
    diaids_val = diaids_merged[train_idx:val_idx]
    diaids_test = diaids_merged[val_idx:]

    assert len(diaids_merged) == (
        len(diaids_train) + len(diaids_val) + len(diaids_test)
    )

    diaids_merged = {"train": diaids_train, "val": diaids_val, "test": diaids_test}

    utterance_ordered_merged = {}

    for SPLIT in ["train", "val", "test"]:
        utterance_ordered_merged[SPLIT] = {}

        for diaid in tqdm(diaids_merged[SPLIT]):

            d_, s_, d__ = diaid.split("/")
            utterance_ordered_merged[SPLIT][diaid] = [
                f"{d_}/{s_}/{d__}/{uttid}" for uttid in utterance_ordered[d_][s_][d__]
            ]

    assert len(
        [
            val___
            for key, val in utterance_ordered.items()
            for key_, val_ in val.items()
            for key__, val__ in val_.items()
            for val___ in val__
        ]
    ) == len(
        [
            val__
            for key, val in utterance_ordered_merged.items()
            for key_, val_ in val.items()
            for val__ in val_
        ]
    )

    with open("./utterance-ordered-MELD_IEMOCAP.json", "w") as stream:
        json.dump(utterance_ordered_merged, stream, indent=4)


def get_num_classes(DATASET: str) -> int:
    """Get the number of classes to be classified by dataset."""
    if DATASET == "MELD":
        NUM_CLASSES = 7
    elif DATASET == "IEMOCAP":
        NUM_CLASSES = 6
    elif DATASET == "MELD_IEMOCAP":
        NUM_CLASSES = 7
    else:
        raise ValueError

    return 7


def compute_metrics(eval_predictions) -> dict:
    """Return f1_weighted, f1_micro, and f1_macro scores."""
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)

    f1_weighted = f1_score(label_ids, preds, average="weighted")
    f1_micro = f1_score(label_ids, preds, average="micro")
    f1_macro = f1_score(label_ids, preds, average="macro")

    return {"f1_weighted": f1_weighted, "f1_micro": f1_micro, "f1_macro": f1_macro}


def set_seed(seed: int) -> None:
    """Set random seed to a fixed value.

    Set everything to be deterministic
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


def get_emotion2id(DATASET: str) -> Tuple[dict, dict]:
    """Get a dict that converts string class to numbers."""
    emotions = [
        "neutral",
        "joy",
        "surprise",
        "anger",
        "sadness",
        "disgust",
        "fear",
    ]
        
    emotion2id = {
        "neutral": 0,
        "joy": 1,
        "happiness": 1,
        "excited": 1,
        "surprise": 2,
        "anger": 3,
        "frustration": 3,
        "sadness": 4,
        "disgust": 5,
        "fear": 6,
    }
    id2emotion = {idx: emotion for idx, emotion in enumerate(emotions)}

    return emotion2id, id2emotion


class ErcTextDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        DATASET="MELD",
        SPLIT="train",
        speaker_mode="upper",
        num_past_utterances=0,
        num_future_utterances=0,
        model_checkpoint="roberta-base",
        ROOT_DIR="./",
        ONLY_UPTO=False,
        SEED=0,
    ):
        """Initialize emotion recognition in conversation text modality dataset class."""

        self.DATASET = DATASET
        self.ROOT_DIR = ROOT_DIR
        self.SPLIT = SPLIT
        self.speaker_mode = speaker_mode
        self.num_past_utterances = num_past_utterances
        self.num_future_utterances = num_future_utterances
        self.model_checkpoint = model_checkpoint
        self.emotion2id, self.id2emotion = get_emotion2id(self.DATASET)
        self.ONLY_UPTO = ONLY_UPTO
        self.SEED = SEED

        self._load_emotions()
        self._load_utterance_ordered()
        self._string2tokens()

    def _load_emotions(self):
        """Load the supervised labels"""
        if self.DATASET in ["MELD", "IEMOCAP"]:
            with open(
                os.path.join(self.ROOT_DIR, self.DATASET, "emotions.json"), "r"
            ) as stream:
                self.emotions = json.load(stream)[self.SPLIT]

    def _load_utterance_ordered(self):
        """Load the ids of the utterances in order."""
        if self.DATASET in ["MELD", "IEMOCAP"]:
            path = os.path.join(self.ROOT_DIR, self.DATASET, "utterance-ordered.json")
        elif self.DATASET == "MELD_IEMOCAP":
            path = "./utterance-ordered-MELD_IEMOCAP.json"

        with open(path, "r") as stream:
            self.utterance_ordered = json.load(stream)[self.SPLIT]

    def __len__(self):
        return len(self.inputs_)

    def _load_utterance_speaker_emotion(self, uttid, speaker_mode) -> dict:
        """Load an speaker-name prepended utterance and emotion label"""

        if self.DATASET in ["MELD", "IEMOCAP"]:
            text_path = os.path.join(
                self.ROOT_DIR, self.DATASET, "raw-texts", self.SPLIT, uttid + ".json"
            )
        elif self.DATASET == "MELD_IEMOCAP":
            assert len(uttid.split("/")) == 4
            d_, s_, d__, u_ = uttid.split("/")
            text_path = os.path.join(self.ROOT_DIR, d_, "raw-texts", s_, u_ + ".json")

        with open(text_path, "r") as stream:
            text = json.load(stream)

        utterance = text["Utterance"].strip()
        emotion = text["Emotion"]

        if self.DATASET == "MELD":
            speaker = text["Speaker"]
        elif self.DATASET == "IEMOCAP":
            sessid = text["SessionID"]
            # https: // www.ssa.gov/oact/babynames/decades/century.html
            speaker = {
                "Ses01": {"Female": "Mary", "Male": "James"},
                "Ses02": {"Female": "Patricia", "Male": "John"},
                "Ses03": {"Female": "Jennifer", "Male": "Robert"},
                "Ses04": {"Female": "Linda", "Male": "Michael"},
                "Ses05": {"Female": "Elizabeth", "Male": "William"},
            }[sessid][text["Speaker"]]
        elif self.DATASET == "MELD_IEMOCAP":
            speaker = ""
        else:
            raise ValueError(f"{self.DATASET} not supported!!!!!!")

        if speaker_mode is not None and speaker_mode.lower() == "upper":
            utterance = speaker.upper() + ": " + utterance
        elif speaker_mode is not None and speaker_mode.lower() == "title":
            utterance = speaker.title() + ": " + utterance

        return {"Utterance": utterance, "Emotion": emotion}

    def _create_input(
        self, diaids, speaker_mode, num_past_utterances, num_future_utterances
    ):
        """Create an input which will be an input to RoBERTa."""

        args = {
            "diaids": diaids,
            "speaker_mode": speaker_mode,
            "num_past_utterances": num_past_utterances,
            "num_future_utterances": num_future_utterances,
        }

        logging.debug(f"arguments given: {args}")
        tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint, use_fast=True)
        max_model_input_size = 128
        num_truncated = 0

        inputs = []
        for diaid in tqdm(diaids):
            ues = [
                self._load_utterance_speaker_emotion(uttid, speaker_mode)
                for uttid in self.utterance_ordered[diaid]
            ]

            num_tokens = [len(tokenizer(ue["Utterance"])["input_ids"]) for ue in ues]

            for idx, ue in enumerate(ues):
                if ue["Emotion"] not in list(self.emotion2id.keys()):
                    continue

                label = self.emotion2id[ue["Emotion"]]

                indexes = [idx]
                indexes_past = [
                    i for i in range(idx - 1, idx - num_past_utterances - 1, -1)
                ]
                indexes_future = [
                    i for i in range(idx + 1, idx + num_future_utterances + 1, 1)
                ]

                offset = 0
                if len(indexes_past) < len(indexes_future):
                    for _ in range(len(indexes_future) - len(indexes_past)):
                        indexes_past.append(None)
                elif len(indexes_past) > len(indexes_future):
                    for _ in range(len(indexes_past) - len(indexes_future)):
                        indexes_future.append(None)

                for i, j in zip(indexes_past, indexes_future):
                    if i is not None and i >= 0:
                        indexes.insert(0, i)
                        offset += 1
                        if (
                            sum([num_tokens[idx_] for idx_ in indexes])
                            > max_model_input_size
                        ):
                            del indexes[0]
                            offset -= 1
                            num_truncated += 1
                            break
                    if j is not None and j < len(ues):
                        indexes.append(j)
                        if (
                            sum([num_tokens[idx_] for idx_ in indexes])
                            > max_model_input_size
                        ):
                            del indexes[-1]
                            num_truncated += 1
                            break

                utterances = [ues[idx_]["Utterance"] for idx_ in indexes]

                if num_past_utterances == 0 and num_future_utterances == 0:
                    assert len(utterances) == 1
                    final_utterance = utterances[0]

                elif num_past_utterances > 0 and num_future_utterances == 0:
                    if len(utterances) == 1:
                        final_utterance = utterances[-1]
                    else:
                        final_utterance = (
                            " ".join(utterances[:-1]) + " [CTX] " + utterances[-1]
                        )

                elif num_past_utterances == 0 and num_future_utterances > 0:
                    if len(utterances) == 1:
                        final_utterance = utterances[0] + " [CTX] "
                    else:
                        final_utterance = (
                            utterances[0] + " [CTX] " + " ".join(utterances[1:])
                        )

                elif num_past_utterances > 0 and num_future_utterances > 0:
                    if len(utterances) == 1:
                        final_utterance = utterances[0]
                    else:
                        final_utterance = (
                            " ".join(utterances[:offset])
                            + " [CTX] "
                            + utterances[offset]
                            + " [CTX] "
                            + " ".join(utterances[offset + 1 :])
                        )
                else:
                    raise ValueError

                #input_ids_attention_mask = tokenizer(final_utterance)
                #input_ids = input_ids_attention_mask["input_ids"]
                #attention_mask = input_ids_attention_mask["attention_mask"]

                input_ = {
                    "id": self.utterance_ordered[diaid][idx],
                    "utterance": final_utterance,
                    "label": self.id2emotion[label],
                }

                inputs.append(input_)

        logging.info(f"number of truncated utterances: {num_truncated}")
        return inputs

    def _string2tokens(self):
        """Convert string to (BPE) tokens."""
        logging.info(f"converting utterances into tokens ...")

        diaids = sorted(list(self.utterance_ordered.keys()))

        set_seed(self.SEED)
        random.shuffle(diaids)

        if self.ONLY_UPTO:
            logging.info(f"Using only the first {self.ONLY_UPTO} dialogues ...")
            diaids = diaids[: self.ONLY_UPTO]

        logging.info(f"creating input utterance data ... ")
        self.inputs_ = self._create_input(
            diaids=diaids,
            speaker_mode=self.speaker_mode,
            num_past_utterances=self.num_past_utterances,
            num_future_utterances=self.num_future_utterances,
        )

    def __getitem__(self, index):

        return self.inputs_[index]

In [2]:
"""
class ErcTextDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        DATASET="MELD",
        SPLIT="train",
        speaker_mode="upper",
        num_past_utterances=0,
        num_future_utterances=0,
        model_checkpoint="roberta-base",
        ROOT_DIR="./",
        ONLY_UPTO=False,
        SEED=0,
    ):
"""

'\nclass ErcTextDataset(torch.utils.data.Dataset):\n    def __init__(\n        self,\n        DATASET="MELD",\n        SPLIT="train",\n        speaker_mode="upper",\n        num_past_utterances=0,\n        num_future_utterances=0,\n        model_checkpoint="roberta-base",\n        ROOT_DIR="./",\n        ONLY_UPTO=False,\n        SEED=0,\n    ):\n'

In [3]:
ds_meld_train = ErcTextDataset(DATASET="MELD", SPLIT="train", speaker_mode=None, num_past_utterances=2, num_future_utterances=0)

2023-05-12 01:45:26.957 INFO 3693349904 - _string2tokens: converting utterances into tokens ...
2023-05-12 01:45:26.959 INFO 3693349904 - _string2tokens: creating input utterance data ... 
100%|██████████████████████████████████████| 1038/1038 [00:01<00:00, 708.98it/s]
2023-05-12 01:45:29.952 INFO 3693349904 - _create_input: number of truncated utterances: 1


In [4]:
ds_meld_val = ErcTextDataset(DATASET="MELD", SPLIT="val", speaker_mode=None, num_past_utterances=2, num_future_utterances=0)

2023-05-12 01:45:29.967 INFO 3693349904 - _string2tokens: converting utterances into tokens ...
2023-05-12 01:45:29.968 INFO 3693349904 - _string2tokens: creating input utterance data ... 
100%|████████████████████████████████████████| 114/114 [00:00<00:00, 662.60it/s]
2023-05-12 01:45:31.745 INFO 3693349904 - _create_input: number of truncated utterances: 0


In [5]:
ds_meld_test = ErcTextDataset(DATASET="MELD", SPLIT="test", speaker_mode=None, num_past_utterances=2, num_future_utterances=0)

2023-05-12 01:45:31.758 INFO 3693349904 - _string2tokens: converting utterances into tokens ...
2023-05-12 01:45:31.759 INFO 3693349904 - _string2tokens: creating input utterance data ... 
100%|████████████████████████████████████████| 280/280 [00:00<00:00, 756.76it/s]
2023-05-12 01:45:33.109 INFO 3693349904 - _create_input: number of truncated utterances: 0


In [6]:
ds_iemocap_train = ErcTextDataset(DATASET="IEMOCAP", SPLIT="train", speaker_mode=None, num_past_utterances=2, num_future_utterances=0)

2023-05-12 01:45:33.121 INFO 3693349904 - _string2tokens: converting utterances into tokens ...
2023-05-12 01:45:33.121 INFO 3693349904 - _string2tokens: creating input utterance data ... 
100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 84.06it/s]
2023-05-12 01:45:36.279 INFO 3693349904 - _create_input: number of truncated utterances: 58


In [7]:
ds_iemocap_val = ErcTextDataset(DATASET="IEMOCAP", SPLIT="val", speaker_mode=None, num_past_utterances=2, num_future_utterances=0)

2023-05-12 01:45:36.290 INFO 3693349904 - _string2tokens: converting utterances into tokens ...
2023-05-12 01:45:36.291 INFO 3693349904 - _string2tokens: creating input utterance data ... 
100%|███████████████████████████████████████████| 20/20 [00:00<00:00, 75.29it/s]
2023-05-12 01:45:37.915 INFO 3693349904 - _create_input: number of truncated utterances: 20


In [8]:
ds_iemocap_test = ErcTextDataset(DATASET="IEMOCAP", SPLIT="test", speaker_mode=None, num_past_utterances=2, num_future_utterances=0)

2023-05-12 01:45:37.925 INFO 3693349904 - _string2tokens: converting utterances into tokens ...
2023-05-12 01:45:37.926 INFO 3693349904 - _string2tokens: creating input utterance data ... 
100%|███████████████████████████████████████████| 31/31 [00:00<00:00, 72.20it/s]
2023-05-12 01:45:40.013 INFO 3693349904 - _create_input: number of truncated utterances: 33


In [9]:
import pandas as pd

2023-05-12 01:45:40.053 INFO utils - _init_num_threads: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-05-12 01:45:40.053 INFO utils - _init_num_threads: NumExpr defaulting to 8 threads.


In [10]:
df_meld_train = pd.DataFrame(ds_meld_train.inputs_)
df_meld_train = df_meld_train.assign(path=df_meld_train.id.apply(lambda x: f"multimodal-datasets/MELD/raw-audios/train_resampled/{x}.wav"))

df_meld_val = pd.DataFrame(ds_meld_val.inputs_)
df_meld_val = df_meld_val.assign(path=df_meld_val.id.apply(lambda x: f"multimodal-datasets/MELD/raw-audios/val_resampled/{x}.wav"))

df_meld_test = pd.DataFrame(ds_meld_test.inputs_)
df_meld_test = df_meld_test.assign(path=df_meld_test.id.apply(lambda x: f"multimodal-datasets/MELD/raw-audios/test_resampled/{x}.wav"))

df_iemocap_train = pd.DataFrame(ds_iemocap_train.inputs_)
df_iemocap_train = df_iemocap_train.assign(path=df_iemocap_train.id.apply(lambda x: f"multimodal-datasets/IEMOCAP/raw-audios/train/{x}.wav"))

df_iemocap_val = pd.DataFrame(ds_iemocap_val.inputs_)
df_iemocap_val = df_iemocap_val.assign(path=df_iemocap_val.id.apply(lambda x: f"multimodal-datasets/IEMOCAP/raw-audios/val/{x}.wav"))

df_iemocap_test = pd.DataFrame(ds_iemocap_test.inputs_)
df_iemocap_test = df_iemocap_test.assign(path=df_iemocap_test.id.apply(lambda x: f"multimodal-datasets/IEMOCAP/raw-audios/test/{x}.wav"))

In [11]:
train_df = pd.concat([df_meld_train.drop(columns=["id"]), df_iemocap_train.drop(columns=["id"]), df_iemocap_val.drop(columns=["id"])]).reset_index(drop=True)
train_df

,utterance,label,path
0,"OK, we got the cole slaw, we got the buns...",neutral,multimodal-datasets/MELD/raw-audios/train_resa...
1,"OK, we got the cole slaw, we got the buns... [...",disgust,multimodal-datasets/MELD/raw-audios/train_resa...
2,"OK, we got the cole slaw, we got the buns... W...",neutral,multimodal-datasets/MELD/raw-audios/train_resa...
3,"Wait a minute, hold it. Johnson! Will you com...",neutral,multimodal-datasets/MELD/raw-audios/train_resa...
4,"Wait a minute, hold it. Johnson! Will you com...",neutral,multimodal-datasets/MELD/raw-audios/train_resa...
...,...,...,...
15863,"Right. Um, Thanks for asking, though. I mean, ...",neutral,multimodal-datasets/IEMOCAP/raw-audios/val/Ses...
15864,"God, that's like a whole problem with like-- B...",neutral,multimodal-datasets/IEMOCAP/raw-audios/val/Ses...
15865,"if you ever come out to Montgomery, I mean, yo...",neutral,multimodal-datasets/IEMOCAP/raw-audios/val/Ses...
15866,"call up the Sprint call center and, uh, you kn...",neutral,multimodal-datasets/IEMOCAP/raw-audios/val/Ses...


In [12]:
test_df = pd.concat([df_iemocap_test.drop(columns=["id"]), df_meld_val.drop(columns=["id"])]).reset_index(drop=True)
test_df

,utterance,label,path
0,"Brian, I need help.",sadness,multimodal-datasets/IEMOCAP/raw-audios/test/Se...
1,"Brian, I need help. [CTX] Babe, I don't know w...",neutral,multimodal-datasets/IEMOCAP/raw-audios/test/Se...
2,"Babe, I don't know what to tell you. Don't gi...",neutral,multimodal-datasets/IEMOCAP/raw-audios/test/Se...
3,I don't--I'm just--I'm thinking maybe I should...,anger,multimodal-datasets/IEMOCAP/raw-audios/test/Se...
4,"Well, of course not, but what are you going to...",anger,multimodal-datasets/IEMOCAP/raw-audios/test/Se...
...,...,...,...
2754,Cantaloupe juice. Exactly. [CTX] You've tasted...,surprise,multimodal-datasets/MELD/raw-audios/val_resamp...
2755,Exactly. You've tasted it? You've tasted it. [...,neutral,multimodal-datasets/MELD/raw-audios/val_resamp...
2756,You've tasted it? You've tasted it. Uh huh. [C...,anger,multimodal-datasets/MELD/raw-audios/val_resamp...
2757,"Uh huh. Oh, you've tasted it. [CTX] You can ke...",neutral,multimodal-datasets/MELD/raw-audios/val_resamp...


In [13]:
import os

In [14]:
not_file = []
for i, row in train_df.iterrows():
    if not os.path.isfile(f"../{row['path']}"):
        not_file.append(row['path'])

In [15]:
#not_file = []
for i, row in test_df.iterrows():
    if not os.path.isfile(f"../{row['path']}"):
        not_file.append(row['path'])

In [16]:
not_file

['multimodal-datasets/MELD/raw-audios/train_resampled/dia125_utt3.wav',
 'multimodal-datasets/MELD/raw-audios/val_resampled/dia110_utt7.wav']

In [17]:
len(train_df)

15868

In [18]:
train_df = train_df[train_df["path"] != 'multimodal-datasets/MELD/raw-audios/train_resampled/dia125_utt3.wav']

In [19]:
len(train_df)

15867

In [20]:
len(test_df)

2759

In [21]:
test_df = test_df[test_df["path"] != 'multimodal-datasets/MELD/raw-audios/val_resampled/dia110_utt7.wav']

In [22]:
len(test_df)

2758

In [23]:
train_df

,utterance,label,path
0,"OK, we got the cole slaw, we got the buns...",neutral,multimodal-datasets/MELD/raw-audios/train_resa...
1,"OK, we got the cole slaw, we got the buns... [...",disgust,multimodal-datasets/MELD/raw-audios/train_resa...
2,"OK, we got the cole slaw, we got the buns... W...",neutral,multimodal-datasets/MELD/raw-audios/train_resa...
3,"Wait a minute, hold it. Johnson! Will you com...",neutral,multimodal-datasets/MELD/raw-audios/train_resa...
4,"Wait a minute, hold it. Johnson! Will you com...",neutral,multimodal-datasets/MELD/raw-audios/train_resa...
...,...,...,...
15863,"Right. Um, Thanks for asking, though. I mean, ...",neutral,multimodal-datasets/IEMOCAP/raw-audios/val/Ses...
15864,"God, that's like a whole problem with like-- B...",neutral,multimodal-datasets/IEMOCAP/raw-audios/val/Ses...
15865,"if you ever come out to Montgomery, I mean, yo...",neutral,multimodal-datasets/IEMOCAP/raw-audios/val/Ses...
15866,"call up the Sprint call center and, uh, you kn...",neutral,multimodal-datasets/IEMOCAP/raw-audios/val/Ses...


In [24]:
test_df

,utterance,label,path
0,"Brian, I need help.",sadness,multimodal-datasets/IEMOCAP/raw-audios/test/Se...
1,"Brian, I need help. [CTX] Babe, I don't know w...",neutral,multimodal-datasets/IEMOCAP/raw-audios/test/Se...
2,"Babe, I don't know what to tell you. Don't gi...",neutral,multimodal-datasets/IEMOCAP/raw-audios/test/Se...
3,I don't--I'm just--I'm thinking maybe I should...,anger,multimodal-datasets/IEMOCAP/raw-audios/test/Se...
4,"Well, of course not, but what are you going to...",anger,multimodal-datasets/IEMOCAP/raw-audios/test/Se...
...,...,...,...
2754,Cantaloupe juice. Exactly. [CTX] You've tasted...,surprise,multimodal-datasets/MELD/raw-audios/val_resamp...
2755,Exactly. You've tasted it? You've tasted it. [...,neutral,multimodal-datasets/MELD/raw-audios/val_resamp...
2756,You've tasted it? You've tasted it. Uh huh. [C...,anger,multimodal-datasets/MELD/raw-audios/val_resamp...
2757,"Uh huh. Oh, you've tasted it. [CTX] You can ke...",neutral,multimodal-datasets/MELD/raw-audios/val_resamp...


In [25]:
train_df.reset_index(drop=True).to_csv("train_text_df.csv")

In [26]:
test_df.reset_index(drop=True).to_csv("test_text_df.csv")